# Text-To-Speech inference with my funny API

This notebook illustrates how to use the `tts` module as API to perform inference with your pretrained `Tacotron-2` (or `SV2TTS`) model as `synthesizer`, and the pretrained `NVIDIA's Waveglow` as `vocoder`

The API is easy to use ! Simply call the function with your sentence(s), and the model you want to use, and that's it ! The code automatically loads the right model and makes the whole inference !

It is also possible to associate models to language in the `models/tts/__init__.py` file, in order to enable *language-based* loading (instead of *model-based* loading) (see the 2nd cell for example)

Note that `Vocoder` is loaded as global variable, and all `BaseModel` are `singleton`, so you can call the function as mny times as you want without reloading models every time !

I left the results in the `example_outputs/` directory such that you can listen to them without re-executing the code ;)

Note : it is normal that the 1st generation is relatively slow (because of models' loading and `tensorflow graph` compilation) but the next ones will be faster than real time ! A possible workaround is to call the `tts.load_tts_model(model_name, tf_compile = True)` to pre-compile the model on single-character texts, which makes the compilation faster ;)

PS : I will (*theorically*) **never** share weights for the last French demonstration (at the end of this notebook) (which is a `fine-tuned SV2TTS`), but other French voices trained on `SIWIS` (single-speaker) and `SIWIS + CommonVoice + VoxForge` (multi-speaker) are shared !

## Steps to reproduce

1. Download model weights (see `README.md` for links)
2. Unzip weights in `pretrained_models/` directory
3. Execute cells !

Note : to associate a model to a language, go to `models/tts/__init__.py` and modify the `_pretrained` global variable (at the end of the file). The `key` is the language and the `value` is de model's name.

**Important note** : it is not required to download the `WaveGlow` vocoder model. It is also possible to use the `torch hub` model but it requires a working **GPU-enabled** pytorch installation. If you do not want to have both tensorflow and pytorch working together on GPU, you can download the `WaveGlow` tensorflow implementation I made. 

### Streaming

This functionality allows to enter text and *directly* (up to the inference time) get the output

In [ ]:
from models.tts import tts_stream

# I suggest you to not save audios when streaming because it is slower ;)
# I did it so that you can listen to the result

# PS : the 'goodbye, see you soon !' message at the end is a funny sentence I added
# when you stop the streaming :D
tts_stream(model = 'pretrained_tacotron2', directory = 'example_outputs/streaming', overwrite = True)


### TTS on text

This function generates audios based on the provided text / list of text. 

By default, the model does not regenerate sentences if they have already been generated, but you can change this behavior with the `overwrite` argument to force regeneration. Note that for `SV2TTS`-based models, `overwrite = True` by default as those models are designed to have multiple intonations as they are *multi-speakers* models. 

In this example, the model is loaded with `lang = 'en'`, which loads the `pretrained_tacotron2` model. In the `models/tts/__init__.py` file, it is possible to modify the association by changing the global `_pretrained` variable. 

For `SV2TTS` model, it is possible to specify new custom embeddings by specifying the `embeddings` (and optionnally the `mode`) argument(s). The `embedding` parameter is a 2-D matrix of embeddings (of shape `(n_embeddings, embedding_dim)`) (or a `pd.DataFrame` with an `embedding` column), and is used to control the intonation / voice of the generated audio. The `mode` parameter is used to select the embedding to use (e.g. `mode = 0` will take the 1st embedding of the matrix) (see `utils.select_embedding` for more information). 

In [ ]:
from models.tts import tts
from loggers import set_level

text = """
Hello world ! I hope you will enjoy this funny API for Text-To-Speech ! 
If you train new models, do not hesitate to contact me, or add it in the available models !
"""

set_level('info')

_ = tts(
    text, lang = 'en', directory = 'example_outputs/en', display = True, overwrite = True
)

In [ ]:
from models.tts import tts

text = [
    "Bonjour tout le monde ! J'espère que vous allez aimer cette démonstration de voix en français !"
]

_ = tts(
    text, lang = 'fr', directory = 'example_outputs/fr', display = True,  overwrite = True
)

In regular inference, `Tacotron-2` (and `SV2TTS`) models are not able to predict on too long texts (i.e. longer than around 150 caracters), due to their attention mechanism. To solve this issue, the `attn_mask_win_len` will dynamically move a sliding window to keep at most the correct number of tokens visible by the attention. It is therefore important to correctly set this parameter when your text is too long, or use the `max_text_length` argument to split the text. The 2nd solution works well, but gives results of lower quality as the model do not have access to the entire text, and will therefore not produce a *smooth and continuous* reading of the text. Nonetheless, the masking feature is still experimental and may fail in some cases ;) Thanks to the randomness nature of the model, a simple workaround is to re-execute the generation (with `overwrite = True`) to have another result !

In [ ]:
from models.tts import tts

text = """
Bonjour tout le monde ! J'espère que vous allez aimer cette démonstration de mon super modèle entrainé avec seulement 20 minutes 
d'audios ! Je ne partagerai pas ce modèle, mais je trouvais ça intéressant de montrer ce qu'il était possible de faire !
"""

_ = tts(
    text, model = 'sv2tts_fine_tuned', directory = 'example_outputs/fr', attn_mask_win_len = 150, display = True, overwrite = False
)

This cell displays an audio based on its saving directory and the text. The text is saved in its *cleaned* form, meaning that this original (uncleaned) text will not match any saved audio. It is the reason why the `model` argument is added, in order to load its `TextEncoder`, cleans the text and finally gets it. The `add_model_name` is also required to not add the model's name at the end of the directory.

In [ ]:
from models.tts import get_audio_file
from utils.audio import display_audio

text1 = "Bonjour tout le monde ! J'espère que vous allez aimer cette démonstration de mon super modèle entrainé avec seulement 20 minutes d'audios ! \
Je ne partagerai pas ce modèle, mais je trouvais ça intéressant de montrer ce qu'il était possible de faire !"

text2 = "Bonjour tout le monde ! J'espère que vous allez aimer cette démonstration de mon super modèle entrainé avec seulement 5 minutes d'audios !\
Je ne partagerai pas cemodèle, mais je trouvais ça intéressant de montrer ce qu'il était possible de faire !"

_ = display_audio(get_audio_file(text1, directory = 'example_outputs/fr', model = 'sv2tts_siwis_v2', add_model_name = False))

#_ = display_audio(get_audio_file(text2, directory = 'example_outputs/fr', model = 'sv2tts_siwis_v3', add_model_name = False))